### RAG - Graphe de connaissance

#### Initialisation du graphe avec ses données

Le KG est un exemple simplifié des objets et relations typique d'un outil de SDLC/Devops (team, tasks, service, etc.)

<img src="./assets/kag.png" alt="drawing" width="400"/>

In [ ]:
import dotenv
dotenv.load_dotenv()

from langchain_community.graphs import Neo4jGraph
import json

graph = Neo4jGraph()

with open('./docs/microservices.json') as json_file:
    data = json.load(json_file)
    print(f"data: \n {data}")
    graph.query(data['query'])

: 

#### Initialisation du graphe

Nous utilisons `langchain` pour peupler créer un vector store sur la propriété `description`des noeuds `Task`du graphe

[`Neo4j`](http://localhost:7474/browser/) 

In [ ]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

#### Recherche de similarité et chat

Le vector store est maintenant disponible pour de la recherche de similarité sur la description des Task.

In [ ]:
response = vector_index.similarity_search(
    "How will RecommendationService be updated?"
)
print(response[0].page_content)

De la même manière nous pouvons maintenant utiliser notre vecteur store dans le KG pour alimenter le LLM.

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(), chain_type="stuff", retriever=vector_index.as_retriever())

vector_qa.invoke(
    {"query": "How will recommendation service be updated?"}
)

##### Les limites du RAG simple (naïf?)

In [ ]:
vector_qa.invoke(
    {"query": "Combien y-a-t-il de ticket Open?"}
)

Le LLM est affirmatif mais la réponse est incorrecte !

Ce n'est pas une hallucination du LLM mais une limite de notre RAG.
En effet la qualité de la réponse du LLM est directement corrélé à la qualité et quantité des informations pertinentes trouvées dans le vector store. Ici notre chaîne est paramétrée par défaut pour envoyer au LLM les 4 documents les plus pertinents.

#### Utiliser le LLM pour produire directement une requête sur le KG

Langchain fournit une chaîne préconfigurée avec un agent disposant d'un Tool lui permettant de générer et exécuter des requêtes sur le Knowledge Graph.

In [ ]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4'),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [ ]:
cypher_chain.invoke(
    {"query": "Combien y-a-t-il de ticket ouverts??"}
)


In [ ]:

cypher_chain.invoke(
    {"query": "Which services depend on Database indirectly?"}
)

#### Mixer RAG et Knowledge graph dans un Agent

In [ ]:
from langchain.agents import create_openai_functions_agent, Tool, AgentExecutor
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate

tools = [
    Tool(
        name="Tasks",
        func=vector_qa.invoke,
        description="""Useful when you need to answer questions about descriptions of tasks.
        Not useful for counting the number of tasks.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.invoke,
        description="""Useful when you need to answer questions about microservices,
        their dependencies or assigned people. Also useful for any sort of
        aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

prompt = prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant"),
  ("placeholder", "{chat_history}"),
  ("human", "{input}"),
  ("placeholder", "{agent_scratchpad}")]
)
agent = create_openai_functions_agent(
    ChatOpenAI(temperature=0, model_name='gpt-4'), tools, prompt
)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke({"input": "Which team is assigned to maintain PaymentService?"})

In [ ]:
response = agent_executor.invoke({"input": "Which tasks have optimization in their description?"})